# bigram model

> Let's create a simple bigram model for text generation based on the Tiny Shakespear dataset

In [ ]:
#| default_exp bigram

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

We will be using PyTorch for this model:

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

Hyperparameters:

In [ ]:
batch_size = 32 # how many sequences will we process in parallel?
block_size = 8 # what is the maximum context length?
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu' # use cuda if available
eval_iters = 200
n_embd = 32
torch.manual_seed(1337)

<torch._C.Generator>

Getting the Tiny Shakepeare dataset (a small text file containing all the works of Shakepeare):

In [ ]:
!wget -O /home/nclark/git/framework_gpt/nbs/data/input.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

with open('/home/nclark/git/framework_gpt/nbs/data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

--2023-02-09 14:46:56--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘/home/nclark/git/framework_gpt/nbs/data/input.txt’

/home/nclark/git/fr 100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2023-02-09 14:46:56 (7.11 MB/s) - ‘/home/nclark/git/framework_gpt/nbs/data/input.txt’ saved [1115394/1115394]



Vocabulary Creation and Encoder:

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] #encoder: string -> list of integers
decode = lambda l: ''.join(itos[i] for i in l) #decoder: list of integers -> string

90/10 split into training and validation sets

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

Data Loading:

In [ ]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data =train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

Create an estimate_loss function for evaluation while training:

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

Bigram Model:

In [ ]:
class Head(nn.Module):
    """one head of self-attention"""
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size,bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        # computer attention scores
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, 16) @ (B, 16, T) ---> (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T)
        wei = F.softmax(wei, dim=-1) #(B,T,T)
        # weighted aggregation of values
        v = self.value(x) #(B,T,C)
        out = wei @ v #(B,T,T) @ (B,T,C) _> (B,T,C)
        return out

In [ ]:
class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""
    
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        
    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim=-1)

In [ ]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        # each token reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_heads =MultiHeadAttention(4, n_embd//4) #4 heads of 8-dimensional self-attention
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        # idx and targets are both (B, T) tensors of integers
        tok_emb = self.token_embedding_table(idx) #(B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.sa_heads(x) #apply one head of self-attention
        logits = self.lm_head(x) #(B,T,vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get predicions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

Initialize Model:

In [ ]:
model = BigramLanguageModel()
m = model.to(device)

Optimizer and Training Loop:

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):
    
    # evaluate the loss on train and val sets every eval_interval
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    #evluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.2933, val loss 4.2968
step 500: train loss 2.6631, val loss 2.6575
step 1000: train loss 2.5354, val loss 2.5247
step 1500: train loss 2.4781, val loss 2.4689
step 2000: train loss 2.4364, val loss 2.4388
step 2500: train loss 2.3951, val loss 2.4072
step 3000: train loss 2.3656, val loss 2.3884
step 3500: train loss 2.3505, val loss 2.3596
step 4000: train loss 2.3166, val loss 2.3502
step 4500: train loss 2.3141, val loss 2.3229


Generate Text:

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))



Wherre chey thark,
Andinfe thand ther, we oto.

The hin.

ABcat
R :
LE:
K:
Wheline lillo anerd fol othouadbe nd th:
Oe be.
QEUELORYo word wheo?

Wyor I:
I th aus Euld't ceryondm's ourrse;
Hansas sthe Ecokrnd.
BUSu tour tt eslf bllle dirord ht wacetea yous, PEE:
Coutr withorm sande, fo st othe hd tter yos I cubne seg min:
Ther'liy houre er
I ate itertohorudr, Bulded the'dd En RWH:
Wy,
IWees sbece fount CIHICARLCENTOD BRE:
CAthe kallden thit.

Ad sondno ha quy, I:
Cubre ellids bles dalato frer th
